In [1]:
import os
import re
import time
import jieba
import traceback
import numpy as np
import pandas as pd

from tqdm import tqdm
from itertools import chain
from collections import defaultdict
from multiprocessing import Pool
# from multiprocessing import get_context
from functools import partial

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
pd.set_option('display.max_rows', 10**5)
pd.set_option('display.max_colwidth', -1)

---

# Load

In [4]:
def load(path, filename):

    df = pd.read_csv(os.path.join(path, filename), index_col=0)
    # print(df.shape)
    # df.head(1)

    texts = df[['content']]
    # print(texts.shape)
    # texts.head(1)
    
    return texts

In [5]:
# texts = texts[texts.content == '高价收吴国高迁号，有意的私聊，晚上九点统一回复！']
# texts = texts[texts.content == '有意请加VX18526109947']
# texts

---

# Chars

## localization

In [6]:
def filter_localization(texts):

    texts = texts[~texts['content'].str.contains('^{localization:[0-9]+\-[0-9]+}$')].reset_index(drop=True)
    # print(texts.shape)
    # texts.head(1)
    
    return texts

## battle

In [7]:
def filter_battle(texts):

    texts = texts[~texts['content'].str.contains('^{battle:[0-9]+,【.*】.*}$')].reset_index(drop=True)
    # texts.shape

    return texts

## system info

In [8]:
def filter_system_info(texts):

    texts = texts[~texts['content'].str.contains('^{[a-z]*:[0-9]+.*}$')].reset_index(drop=True)
    # texts.shape

    return texts

## clean

In [9]:
def clean_lower(texts):

    texts['tokens'] = texts['content'].str.strip().str.lower()
    # print(len(texts))

    return texts

In [10]:
def clean_special_words(texts):

    sw = [r'\s+', r'{localization:[0-9]+\-[0-9]+}', '丶']

    for _ in sw:
        texts['tokens'] = texts['tokens'].str.replace(_, '')
        # print(len(texts))

    return texts

## meaningless

In [11]:
def clean_numeric_only(texts):

    texts = texts[~texts['tokens'].str.isnumeric()]
    # print(len(texts))

    return texts

## merge chars

In [12]:
def merge_chars(texts):

    dict_merge_chars = {'贝戋' : '贱', '犭句' : '狗', '女马' : '妈'}

    for k, v in dict_merge_chars.items():
        texts['tokens'] = texts['tokens'].str.replace(k, v)

    return texts

---

# Tokens

## tokenization

In [13]:
def tokenization(texts):

    texts['tokens'] = texts['tokens'].apply(jieba.lcut)
    # texts['tokens'] = apply_parallel(texts['tokens'], jieba.lcut)
    # print(len(texts))
    
    # texts['tokens'] = apply_parallel(texts['tokens'], tokenize)

    return texts

## split

### chars + numbers

In [14]:
def chars_numbers(i):
    """ tool
    """
    
    m = re.match('^([a-z]+)([0-9]+)$', i)
    if m: return list(m.groups())
    
    m = re.match('^([0-9]+)([a-z]+)$', i)
    if m: return list(m.groups())
    
    return [i]

In [15]:
def split_chars_numbers(x):
    """ tool
    """
    
    return list(chain.from_iterable([chars_numbers(i) for i in x])) 

In [16]:
def split_cn(texts):

    texts['tokens'] = texts['tokens'].apply(split_chars_numbers)
    # print(len(texts))
    # texts.head(1)
    
    # texts['tokens'] = apply_parallel(texts['tokens'], batch_split_cn)

    return texts

### special words

In [17]:
dict_split = {
'大量出' : ['大量', '出'],
'加微信' : ['加', '微信'],
}

In [18]:
def split_special_words(texts):

    texts['tokens'] = texts['tokens'].apply(lambda x: list(chain.from_iterable([dict_split.get(i,[i]) for i in x])))
    # print(len(texts))
    # texts.head(1)

    return texts

## merge words

In [19]:
list_merge2 = [['资源', '商'], ['张', '辽'], ['c', '位'], ['郡', '城'], ['洗洗', '睡'], ['守', '不住'], ['刚', '有事'], ['弃', '坑'], ['新手', '服'], ['带', '兵'], ['老', '样子'], 
               ['冒个', '泡'], ['白', '嫖'], ['亲', '密度'], ['打寇', '匪'], ['周年', '庆'], ['带', '节奏'], ['玩', '游戏'], ['新', '版本'], ['积点', '德'], ['炸', '矿'], ['赛季', '服'], 
               ['邺', '城'], ['想', '办法'], ['乌', '骓'], ['龟', '孙'], ['红', '手指'], ['死', '妈'], ['2', '队'], ['等', '会'], ['打', '不过'], ['好', '吧'], ['等', '下'], ['打', '掉'], 
               ['去', '吧'], ['等', '一下'], ['打', '一下'], ['打', '哪'], ['收兵', '线'], ['撤', '吧'], ['不', '客气'], ['稍', '等'], ['不', '懂'], ['打', '不了'], ['不会', '吧'],
              ['不', '稳定'], ['下', '一个'], ['人', '不够'], ['努力', '中'], ['赌', '一把'], ['到', '点'], ['1', '队'], ['没看', '懂'], ['+', '1'], ['合个', '影'], ['燕子', '坞'],
              ['一', '梭子']]

In [20]:
list_merge3 = [['青山', '不改', '绿水长流'], ['打', '不', '打'], ['来', '不来']]

In [21]:
def merge3(x):
    """ tool
    """
    
    if 2 > len(x): return x
    
    r = []
    i = len(x) - 1
    
    while(1<i):
        if [x[i-2],x[i-1],x[i]] in list_merge3:
            r.insert(0, ''.join([x[i-2],x[i-1],x[i]]))
            i -= 3
        else:
            r.insert(0, x[i])
            i -= 1

    if 1 == i:
        r.insert(0, x[1])
        r.insert(0, x[0])
            
    return r

In [22]:
def merge2(x):
    """ tool
    """
    
    if 2 > len(x): return x
    
    r = []
    i = len(x) - 1
    
    while(0<i):
        if [x[i-1],x[i]] in list_merge2:
            r.insert(0, ''.join([x[i-1],x[i]]))
            i -= 2
        else:
            r.insert(0, x[i])
            i -= 1
            
    if 0 == i:
        r.insert(0, x[0])
            
    return r

In [23]:
def merge_words_3(texts):

    texts['tokens'] = texts['tokens'].apply(merge3)
    # print(len(texts))
    # texts.head(1)
    
    # texts['tokens'] = apply_parallel(texts['tokens'], batch_merge_words_3)

    return texts

In [24]:
def merge_words_2(texts):

    texts['tokens'] = texts['tokens'].apply(merge2)
    # print(len(texts))
    # texts.head(1)
    
    # texts['tokens'] = apply_parallel(texts['tokens'], batch_merge_words_2)

    return texts

## stopwords

### regular

In [25]:
stopwords = []

### ads detect only

In [26]:
sw = ['…', '⊙', '∀', 'ಡ', 'ω', 'ಡ', '😂', '😜', '谢谢', '哈', '哈哈', '哈哈哈', '哈哈哈哈', '明天', '晚上', '中午', '刷', '刷刷', '霍', '开', '开开', '几点', '没', '改', '军团',
     '散', '结束', '切磋', '队伍', '早', '说', '挂免', '丢人', '打匪', '驻防', '感谢', '再见', '来刷', '抱歉', '行军', '飞', '牛', '牛牛', '做人', '善良', '赞', '赞赞', '赞赞赞',
     '兄弟', '风云', '文明', '飞过来', '拍', '完', '拉', '求虐', '找', '渣', '喊', '拍照', '试试', '谢', '抢', '发', '没事', '喊', '占', '做', '试', '今晚', '更新', '睡觉', 
      '走', '帮忙', '帮', '一会', '吃', '╯', '╰', '溜', '偷', '刷不刷', '←', 'ಥ', '加速', '接', '做', '这是', '呱唧', '先打', '县城', '县', '打不动', '不用', '喝酒', '不好意思',
     '唧唧', '没事', '速度', '先', '派', '弄', '留个', '纪念', 'ง', '•', '̀', '•', '́', '报名', '新', '新版本', '轮子', '嗲', '女马', '一条', '舔', '菊花', '屁股', '秃顶', '后代', '兔子', 
     '鹰酱', '全家', '坑货', '合照', '游戏', '太', '团', '里', '诅咒', '合影', '一队', '二队', '二团', '三团', '一群群', '阿猫', '骗', '删', '统一', '签到', '自动', '老子', '说话', 
      '窝囊废', '死妈', '没妈', '短命', '渣种', '成天', '只会', '亲麻', '打泡', '亲跌', '死', '吠', '骂', '赢', '江湖', '故人', '相逢', '青山', '不改', '绿水长流', '万事', '顺遂',
     '尔隆', '咚锵', '进', '差', '远', '又', '，', '都', '是', '就是', '的', '更好', '发展', '利用', '，', '大家', '一起', '作战', '有', '和', '你', '我', '还', '在线', '。', '*', '、', ':',
     '您', '能', '为', '了', '就', '（', '？', '吗', '来', '了', '能', '不能', '大', '嘛', '被', '那', '在', '！', '他', '再', '自己', '坐标', '截图', '！', '准备', '退场', '2队', '这', 
     '积点德', '也', '让', '他们', '啊', '现在', '为了', '你好', '您好', '本', '有志', '青年', '共同', '迎接', '直接', '申请', '蜀国', '加油', '砸种', '狗', '汗间', '魏主', 
      '祖宗', '现实', '地位', '乐趣', '呢', '认识', '拿', '个', '比', '当', '们', '一场', '持久战', '相信', '胜利', '终将', '属于', '我们', '团结', '一定', '胜利', '欢迎', '所有', 
      '吴国', '大老', '黑', '又称', '真人', '广大', '群众', '远离', '亲码', '嗷嗷叫', '孝顺', '儿', '懆', '得', '这个', '喔', '猪', '拱', '浑身', '溃烂', '正在', '喝', '脓血', '疗伤',
     '可怜', '家里', '得', '剩', '知道', '不', '容易', '父母', '等', '着', '拿', '钱', '买', '棺材', '下葬', '呢', '可是', '那点', '父母', '烧纸', '不够',
      '不', '知道', '回', '云天', '陪', '所谓', '“', '”', '还是', '像', '癞皮狗', '一样', '继续', '留在', '大腿', '可以', '抱', '团里', '当奴才', '拭目以待',
     '种地', '现实', '残酷', '农民', '们', '辛苦', '啦', '不辞辛苦', '坚持', '帽子', '小', '农民', '思维', '真是', '单纯', '可爱', '惹', '人', '疼', '呢', '不', '知道', '不', '打', 
      '宝服', '傻', '笔王', '人',  '呢', '打', '吧', '此', '不肖', '子', '还有', '人', '集结', '留念', '最后', '一期', '好', '凑个', '脸熟', '什么', '意思', '嗯', '好', '情况', '二货',
     '今天', '敌人', '朋友', '这里', '打', '可能', '延迟', '几分钟', '尽量', '保证', '每', '一位', '这个', '时候', '呵呵', '呵', '好', '嘞', '恩怨', '一笑', '泯', '恩仇',
     '啥', '情况', '你们', '哦', '好', '顶战', '清人', '(', ')', '刚', '上线', '我们', '加油', '成员', '全体', '集合', '民心', '任务', '吧', '哪', '认识', '清楚', '么', '分钟', '打不过', 
     '别', '呀', '战力', '以后', '再升', '留点', '兵', '去', '有人', '土匪', '来个', '帮帮忙', '：', '）', '有人', '碰瓷', '厉害', '呀', '怎么', '不理', '因为', '觉得', '咋样',
      '慢慢', '玩吧', '亏', '大发', '别', '互相', '伤害', '告诉', '打野', '有点', '过分', '回家', '公告', '看', '去', '将', '我开', '相忘', '于', '到', '旁边', '乐', '有缘千里', '相会',
     '有缘', '各位', '真', '无聊', '分钟', '我来', '看看', '揍', '没有', '一下', '一个', '上', '下', '用', '不要', '位置', '这么', '繁荣', '谁', '玩', '秒', '把', '多少', '多', '魏国',
      '一二三四五', '过来', '九点', '长安', '次', '不是', '开始', '阵容', '以上', '时间', '很', '清', '带', '城', '打飞', '会', '咯', '地点', '郡城', '留影', '洛阳', '想', '有没有', 
      '搞', '祝', '提前', '随便', '看见', '个人', '起来', '一炮', '不行', '叫', '对', '落位', '上车', '天天', '不好', '令', '分', '区', '高', '王', '其他', '通告', '围城', '联盟',
     '快', '兵线', '地方', '够', '咋', '线', '小时', '跑', '后面', '无', '起', '那个', '蜀', '散人', '郡', '所有人', '本人', '原始', '娱乐', '赶紧', '一点', 'c位', '错', '建筑',
     '成都', '一环', '凑', '希望', '撤', '飞过去', '配合', '其他人', '过', '出兵', '那么', '随机', '最好', '全部', '才', '一直', '领', '中', ',', '可', '现', '开心', '跟', '主公',
     '炸', '马上', '本服', '照', '过节', '大小', '报仇', '三国', '今后', '以前', '消', '弟兄们', '早到', '已经', '自立', '城池', '准时', '种田', '春节', '已放', '下位', '处',
     '方向', '参加', '该', '浪费', '照片', '世界', '见', '一次', '集体', '做个', '到位', '差距', '天', '就位', '打城', '没出息', '奇怪', '好像', '坟头', '不了', '哪个', '不到', 
     '没兵', '彻底', '报', '许昌', '哥哥', '刚刚', '再来', '刚才', '回来', '如果', '出来', '几次', '总是', '应该', '好多', '近', '长沙', '一兵', '不来', '太虚', '随意', '只有',
     '等会', '条渣种', '此时此刻', '被控', '第一', '深更半夜', '平民', '霸服', '脸', '本事', '打出', '别说', '控号', '有钱', '别人', '被控', '残', '狗东西', 
      '地儿', '隆冬', '强', '咚咚', '锵', '整个', '挨个', '群', 'bb', '笔', '内', '间', '内', '畜生', '字', '小名', '童子', '喜', '男人', '男风', '者', '不怕', 'py',
     '狗奴才', '嘴上', '立人', '设时', '连', '当真', '但', '经不起', '任何', '考验', '啪啪', '打脸', '两面三刀', '小人', '晚', '点', '自由', '开放', '1队', '平安', '喜乐', '月',
     '解散', '麻烦', '随迁', '东瀛', '鬼子', '野种', '全飞', '保持', '队形', '寒江', '孤影', '何必', '曾', '相识', '睡', '晚安', '手机', '没电', '123', '茄子', '收兵线', '升', 
      '进步', '很大', '满级', '重', '再也', '不想', '欺负', '弱团', '欺负', '本国', '弱小', '分手', '打掉', '行', '散伙', '臭表子', '一组', '过去', '掉', '全一环', '格子',
     '校场', '副', '再也', '不见', '吓', '跳', '傻笔', '每天', '早点', '休息', '称号', '废', '八姓', '家奴', '几把', '恶是', '几把',
     '赛季', '服前', '一天', '一路', '涪陵', '有空', '按', '顺序', '整体', '排列', '点会', '安排', '大殿', '不带', '协防', '好友', '徒弟', '小车', '死光光',
     '明晚', '长安城', '本区', '提供', '具体地址', '时发', '城到', '铁血', '具体', '吊打', '录屏', '注意', '有刷', '暂时', '离团', '南门', '军旗', '贱狗', '爽', '狗娘养',
    '西南', '门', '军旗', '围', '三环', '抓紧', '骑兵', '进攻',  '抢新', '版本', '首站', '天王', '盖地', '虎', '大白', '卖', '蓝色', '蜀汉', '旗', '大白', '告', '妻',
     '银河', '奥特曼', '沙比', '总喊', '玛', '没钱别', '比比', '能耐', '纳', '闷', '不多', '鱼名', '各路', '英雄', '叛贼', '打家劫舍', '原', '区不分', '国籍', '拍个',
     '钱大狗', '期待', '与', '再次', '相遇', '期', '截止', '想来', '飞行', '玩耍', '万一', '低战图']

In [27]:
sw = list(set(sw))

In [28]:
stopwords.extend(sw)
len(stopwords)

829

In [29]:
def filter_stopwords(texts):

    texts['tokens'] = texts['tokens'].apply(lambda l: [w for w in l if w not in stopwords])
    # texts['tokens'] = apply_parallel(texts['tokens'], lambda l: [w for w in l if w not in stopwords])
    
    # texts['tokens'] = apply_parallel(texts['tokens'], filter_sw)
    # texts = texts.dropna()
    
    
    # print(len(texts))
    # texts.head(1)

    return texts

## repeats

In [30]:
def remove_repeats(x):
    """ tool
    """
    # print(x)
    
    if 2 > len(x): return x
    
    if 2 == len(x): 
        return [x[0]] if x[0] == x[1] else x
    
    r = [x[i] for i in range(len(x)-2) if x[i] != x[i+1]]
    if x[-1] == x[-2]:
        r.append(x[-2])
    else:
        r.extend(x[-2:])
    return r

In [31]:
def filter_repeats(texts):

    # 谢谢, 谢谢
    # 喝酒, 喝酒

    texts['tokens'] = texts['tokens'].apply(remove_repeats)
    # print(len(texts))
    # texts.head(1)

    return texts

## filter

In [32]:
# 'v' + num

def is_v_num(x):
    """ tool
    """
    return 2 == len(x) and 'v' == x[0] and x[1].isnumeric()

In [33]:
def filter_v_num(texts):

    texts = texts[~texts['tokens'].apply(is_v_num)]
    # print(len(texts))

    return texts

In [34]:
# num + '级'

def is_num_ji(x):
    """ tool
    """
    return 2 == len(x) and x[0].isnumeric() and '级' == x[1]

In [35]:
def filter_num_ji(texts):

    texts = texts[~texts['tokens'].apply(is_num_ji)]
    # print(len(texts))

    return texts

## length

In [36]:
def filter_length(texts):
    # texts['length'] = texts['tokens'].apply(len)
    
    texts = texts[texts['tokens'].apply(len) > 1]
    # print(len(texts))
    # print(texts.tail(10))

    return texts

---

# Extract

In [37]:
def extract(texts):

    # df = df.astype(str).value_counts().reset_index().rename(columns={0:'cnt'})
    # df.head(1)

    texts['tokens'] = texts['tokens'].astype(str)
    df = texts.groupby('tokens').agg({'tokens':['count'], 'content':[lambda x: pd.DataFrame.head(x,1)]}).reset_index()

    df.columns = ['tokens', 'cnt', 'content']
    df = df[['tokens','content', 'cnt']]
    df = df.sort_values('cnt', ascending=False).reset_index(drop=True)
    # df.head(1)

    return df

## freq more

In [38]:
def freq_more(df, n=None):

    dfn = df[df['cnt'] >= n]
    # print(dfn.shape)

    return dfn

In [39]:
def freq_between(df, n1=None, n2=None):

    dfn = df[(df['cnt'] >= n1) & (df['cnt'] < n2)]
    # print(dfn.shape)

    return dfn

## keywords

In [40]:
# TODO: 2期 标注

## Data

In [41]:
def extracts(path, keep, filename):

    try:
        
        texts = load(path, filename)
        # texts = texts.head(10)

        pipeline = [
            filter_localization, 
            filter_battle,
            filter_system_info,

            clean_lower,
            clean_special_words,
            clean_numeric_only,
            merge_chars,

            tokenization,
            split_cn,
            split_special_words,
            merge_words_3,
            merge_words_2,
            filter_stopwords,
            filter_repeats,

            filter_v_num,
            filter_num_ji,
            filter_length,
                   ]

        for f in pipeline:
            # print(f)
            t0 = time.time()
            texts = f(texts)
            # print(time.time()-t0)

        df = extract(texts)
        # print(df.shape)

        # dfn = freq_more(df, 5)
        # dfn = freq_between(df, 3, 5)
        # dfn = freq_more(df, 2)
        # dfn = freq_between(df, 1, 2)
        dfn = keep(df)
        print(dfn.shape)
        # print()

        return dfn
    
    except:
        print(filename)
        traceback.print_exc()
        return

In [42]:
# path = '/home/wangyh/data/yk-sgz2017-chat/data-2-20201223'
# filename = '2020_09_25.csv'
# filename = '2020_09_24.csv'
# filename = '2020_08_20.csv'

path = '/home/wangyh/data/yk-sgz2017-chat/data-7-20210120'

In [43]:
def set_similarity(s1, s2):
    _s1 = set(s1)
    _s2 = set(s2)
    return len(_s1 & _s2) / max(len(_s1), len(_s2))

In [44]:
def in_similarity(s1, s):
    for _ in s1:
        # if abs(len(_)-len(c)) < 3 and edit_distance.SequenceMatcher(_, c).ratio() > 0.9:
        if abs(len(_)-len(s)) < 3 and set_similarity(_,s) > 0.9:
            return True
    return False

In [45]:
def get(keep, num=None):

    # split
    # print(len(os.listdir(path)))
    t0 = time.time()
    with Pool(8) as p:
        tasks = os.listdir(path)
        
        # dfs = p.map(partial(extracts, path, keep), tasks)
        
        dfs = []
        for _ in tqdm(p.imap(
            partial(extracts, path, keep), 
            tasks
        ), total=len(tasks)):
            dfs.append(_)
        
    p.join()
    print(time.time() - t0)

    # count
    df_final = pd.concat(dfs).reset_index(drop=True)
    print(df_final.shape)

    df_ = df_final.drop(columns=['cnt'])
    df_ = df_.drop_duplicates('tokens')

    print(df_.shape)

    if num is not None: df_ = df_.head(num)

    # drop duplicates
        
    r = defaultdict(list)
    r[0] = []

    for c in tqdm(df_.content.to_list()):
        flag = True # not exist

        for i in range(max(0,len(c)-3), min(max(r.keys()),len(c)+3)):
            if in_similarity(r[i], c):
                flag = False
                break
                
        if flag:
            r[len(c)].append(c)

    r = list(chain.from_iterable(r.values()))
    print(len(r))

    return pd.DataFrame({'text': r})

In [46]:
df1 = get(partial(freq_more, n=2))
print(df1.shape)
df1.head(3)

  0%|          | 0/107 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.252 seconds.
Prefix dict has been built successfully.
Loading model cost 1.259 seconds.
Prefix dict has been built successfully.
Loading model cost 1.247 seconds.
Prefix dict has b

(71, 3)


  1%|          | 1/107 [00:04<07:09,  4.05s/it]

(71, 3)
(105, 3)


  2%|▏         | 2/107 [00:05<05:30,  3.15s/it]

...

  6%|▌         | 537/9169 [00:00<00:01, 5348.38it/s]

92.94110298156738
(14347, 3)
(9169, 2)


100%|██████████| 9169/9169 [00:26<00:00, 344.07it/s] 


8989
(8989, 1)


,text
0,为了军团更好地发展，资源更好地利用，大家一起作战，兄弟加微信：svip12126，组织打城，军团群里有攻略和内测老人带队，就差你一个人没加了，没加的当机器人踢了
1,为了军团的发展和攻城，大家加下军团长微信：4664632来军团群讨论交流，有阵容攻城攻略，和对换码，一起作战，老人带队，一起发展，以后军团战，没加的当机器人踢了
2,为了军团更好的发展，资源更好的利用，大家一起作战，兄弟加军团长微信：svip12126，领礼包，群里有攻略和内测老人带队，就差你一个人没加，没加的当机器人清理了


In [47]:
df2 = get(partial(freq_between, n1=1, n2=2), num=int(4e4))
print(df2.shape)
df2.head(3)

  0%|          | 0/107 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.275 seconds.
Prefix dict has been built successfully.
Loading model cost 1.265 seconds.
Prefix dict has been built successfully.
Loading model cost 1.269 seconds.
Prefix dict has b

(3745, 3)


  1%|          | 1/107 [00:04<07:08,  4.05s/it]

(3239, 3)
(4221, 3)


  2%|▏         | 2/107 [00:04<05:27,  3.11s/it]

...

  1%|▏         | 565/40000 [00:00<00:06, 5638.52it/s]

(647702, 2)


100%|██████████| 40000/40000 [08:40<00:00, 76.89it/s] 


39891
(39891, 1)


,text
0,你没上兵器
1,没上过学？
2,我没补满兵


In [48]:
df2 = df2[df2.text.apply(lambda x: not in_similarity(df1.text.to_list(), x))]
df2.shape

(39779, 1)

In [49]:
df2['version'] = 2

In [50]:
df2.to_csv('./data/dataset_ads-20210420-2.csv', encoding='utf8', index=False)